In [1]:
import os
os.chdir('../streamlit')

In [2]:
from utils import load_all_data



2025-09-29 15:39:41.763 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-29 15:39:41.765 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-29 15:39:41.765 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-29 15:39:41.766 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-29 15:39:41.767 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-29 15:39:41.767 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-29 15:39:41.768 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-29 15:39:41.769 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-

In [7]:
def add_rankings(df, teg_num):
    """
    Filter to specified TEG and add ranking columns.
    
    Parameters:
    -----------
    df : pandas DataFrame
        The full tournament data from all-data.parquet
    teg_num : int
        The TEG number to filter and rank
    
    Returns:
    --------
    pandas DataFrame
        Filtered data with added TEG_Hole and ranking columns
    """
    # Filter to specified TEG
    df_filtered = df[df['TEGNum'] == teg_num].copy()
    
    # Create cumulative hole number across TEG
    df_filtered['TEG_Hole'] = df_filtered['Hole'] + 18 * (df_filtered['Round'] - 1)
    
    # Round rankings - group by Round and Hole
    df_filtered['Rank_GrossVP_Round'] = df_filtered.groupby(['Round', 'Hole'])['GrossVP Cum Round'].rank(
        method='min', ascending=True, na_option='keep'
    )
    df_filtered['Rank_Stableford_Round'] = df_filtered.groupby(['Round', 'Hole'])['Stableford Cum Round'].rank(
        method='min', ascending=False, na_option='keep'
    )
    
    # TEG rankings - group by TEG_Hole
    df_filtered['Rank_GrossVP_TEG'] = df_filtered.groupby(['TEG_Hole'])['GrossVP Cum TEG'].rank(
        method='min', ascending=True, na_option='keep'
    )
    df_filtered['Rank_Stableford_TEG'] = df_filtered.groupby(['TEG_Hole'])['Stableford Cum TEG'].rank(
        method='min', ascending=False, na_option='keep'
    )
    
    # Reorder columns
    column_order = [
        'Pl', 'Player', 'TEG', 'TEGNum', 'Course', 'Date', 'Round', 'Hole', 'TEG_Hole',
        'Par', 'Sc', 'GrossVP', 'Stableford',
        'GrossVP Cum Round', 'GrossVP Cum TEG', 'Stableford Cum Round', 'Stableford Cum TEG',
        'Rank_GrossVP_Round', 'Rank_GrossVP_TEG', 'Rank_Stableford_Round', 'Rank_Stableford_TEG'
    ]
    
    # Only include columns that exist in the dataframe
    column_order = [col for col in column_order if col in df_filtered.columns]
    return df_filtered[column_order]

    # Add any remaining columns not in our order
    # remaining_cols = [col for col in df_filtered.columns if col not in column_order]
    # final_columns = column_order + remaining_cols
    
    # return df_filtered[final_columns]


# Usage example and CSV export:
# df = pd.read_parquet('all-data.parquet')
# df_with_rankings = add_rankings(df, teg_num=1)
# df_with_rankings.to_csv('teg_rankings.csv', index=False)

In [8]:
df = load_all_data(exclude_incomplete_tegs=False)
df_with_rankings = add_rankings(df, teg_num=17)


In [9]:
df_with_rankings.to_csv('rankings_test.csv', index=False)